In [64]:
import pandas as pd
import numpy as np

In [139]:
data = pd.read_csv('NueBevABTest.csv')

In [7]:
len(data['Client number'].unique())

254

In [8]:
data.shape

(254, 7)

In [140]:
data['Customer Begin'] = pd.to_datetime(data['Customer Begin'])
data['Customer End'] = pd.to_datetime(data['Customer End'], format='%Y-%m-%d')

In [332]:
data['End'] = data['Customer End'].apply(lambda x: x.replace(day=1))
data['Begin'] = data['Customer Begin'].apply(lambda x: x.replace(day=1))

In [45]:
data['Churned'] = data['Churned'].astype('float')

In [54]:
data.head()

,Client number,Region,Customer Begin,Customer End,Churned,Margin Group,Account Size,start_year,start_month,end_year,end_month
0,6025,Los Angeles,2016-06-24,2017-01-27,0.0,High,Medium,2016,6,2017,1
1,7586,Los Angeles,2016-03-10,2017-01-24,0.0,High,Large,2016,3,2017,1
2,9740,Los Angeles,2016-02-26,2016-12-09,1.0,High,Large,2016,2,2016,12
3,1112,Toronto,2016-07-05,2016-10-28,1.0,Low,Small,2016,7,2016,10
4,1223,Toronto,2015-04-27,2017-01-31,0.0,Low,Medium,2015,4,2017,1


In [334]:
dates = [pd.to_datetime('2011-09'), pd.to_datetime('2017-01')]

In [335]:
import datetime
from dateutil.rrule import rrule, MONTHLY

strt_dt = dates[0]
end_dt = dates[1]

dates = [dt for dt in rrule(MONTHLY, dtstart=strt_dt, until=end_dt)]

In [336]:
dates = sorted(dates)

In [364]:
def within_time_period(start, end, target, churned):
    """
    Return true if the target time is between start and end timestamp
    """
    if (target >= start) and (target < end):
        return False
    elif (target == end) and churned == 0:
        return False
    elif (target == end) and churned == 1:
        return True
    else:
        return None

Table: column is each month in the data, each row is the data for that ID

In [341]:
table = pd.DataFrame(columns=dates, index=range(data.shape[0]))

In [342]:
nrow, ncol  = table.shape

In [347]:
# fill the number to the data. 
for i in range(nrow):
    start = data['Begin'][i]
    end = data['End'][i]
    churned = data['Churned'][i]
    for j in range(ncol):
        result = within_time_period(start, end, dates[j], churned)
        table.iloc[i, j] = result

In [346]:
data.head()

,Client number,Region,Customer Begin,Customer End,Churned,Margin Group,Account Size,End,Begin
0,6025,Los Angeles,2016-06-24,2017-01-27,False,High,Medium,2017-01-01,2016-06-01
1,7586,Los Angeles,2016-03-10,2017-01-24,False,High,Large,2017-01-01,2016-03-01
2,9740,Los Angeles,2016-02-26,2016-12-09,True,High,Large,2016-12-01,2016-02-01
3,1112,Toronto,2016-07-05,2016-10-28,True,Low,Small,2016-10-01,2016-07-01
4,1223,Toronto,2015-04-27,2017-01-31,False,Low,Medium,2017-01-01,2015-04-01


In [365]:
rates = np.mean(table, axis = 0)[-5:]

Overall Mean after test began:

In [367]:
np.mean(rates)

0.038851870632770624

# Write up

## Question 2

AB-test makes sense in this case because we are trying to compare the churn rate of two different groups, and AB-test is a method to give a statistical conclusion. 

Another method can be linear regression that studies the relationship between margin level and churned rate. But it is hard for linear regression to study exactly the behaviors of two groups, in our case, margin 15% and 18%. So an AB-test is neccessary. 


## Quesiton 3

To examine the effect of raising the margin, we would run a AB test to our users. We would use the group of users with 15% margin as our control group, and the group of users with higher margin, 18%, as our experiment group. Then we would study whether the churned rate is the same for these two group of users. The type of A/B testing we will use is test of propotion as the question of interest is the churn rate of the two groups. 

The framework of the A/B test will be designed with following steps: 

1. with the table above, calculate the overal average churn rate grouped by two groups. say the two churn rate is p1 and p2 for control group and experiment group, respectively

2. because for each customer, churn or not is a bernouli trial with $\pi$, the variance of the two churn rate is $\frac{\pi*(1-\pi)}{n}$, where n is the number of total client. Therefore, each p follows the normal distribution: $N(\pi, \frac{\pi*(1-\pi)}{n})$

3. set hypothesis 
$$H_{0}: \pi_{1} = \pi_{2}$$
$$H_{1}: \pi_{1} \neq \pi_{2}$$

4. calculate the statistics of the as $\frac{p1-p2}{\frac{p1-(1-p1}{n1}+\frac{p2-(1-p2}{n2}}$ follows N(0,1)

5. finally compare the statistics against a cutoff point or \alpha level to decide reject $$H_{0}$$ or not. 

To avoid affecting users, we group the users into different segments by their location and apply let users from one region all be control or test group. 


The potential rish is that users may realize they are in test and are paying different price for the catering food. 

## Question 4

We would run hypothesis test for a population mean on our data. 

We choose this test because we are trying to study the effect of different margin levels on the churned rate. So we will need to compare the average churned rate between two groups of users of different margin levels. If users with higher margins have higher, or lower churned rate statistically, then we can conclude that margin level does affect churned rate. 

Because we focus on the average value, which is the population mean of two groups, we are using hypothesis test for population mean. 

## Question 5

The null hyputhesis test is that two groups, either with high or low margin level, will have the same churned rate.

## Question 6

Alternative hypothesis will be two groups with different margin level will have different population mean. 

## Question 7

p value

## Question 8

1. as a starting point, we define the effect size as 0.1, which means lower margin group - control group has a 0.1 higher churn rate than experiment group

2. x: likelihood seeing this data, y: effective size. the plot is a decreasing trend